# Import things

In [1]:
import sys
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tnrange, tqdm_notebook

sys.path.insert(0, 'Tools/')
%reload_ext autoreload
%autoreload 2
from data_prep import *

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Load data

In [2]:
overview_selected_files("Data/", "txt")

found files :/57576-0.txt
found files :/57573-0.txt
found files :/57582-0.txt
found files :/57577-0.txt
found files :/57574-0.txt
found files :/57583-0.txt
found files :/57575-0.txt
found files :/57580-0.txt


In [3]:
paths = paths_selected_files("Data/", "txt")

['Data/57576-0.txt', 'Data/57573-0.txt', 'Data/57582-0.txt', 'Data/57577-0.txt', 'Data/57574-0.txt', 'Data/57583-0.txt', 'Data/57575-0.txt', 'Data/57580-0.txt']


In [4]:
book_list = open_book_list(paths)

Book #0 has a word length of 55775
Book #1 has a word length of 788473
Book #2 has a word length of 142117
Book #3 has a word length of 196653
Book #4 has a word length of 265316
Book #5 has a word length of 191917
Book #6 has a word length of 645779
Book #7 has a word length of 623478


# Prepare/clean data

In [5]:
clean_books = [clean_text(book) for book in book_list]


bookstring = ""
for book in clean_books:
    bookstring += book + " "

In [6]:
vocab_to_int = {}
count = 0
for character in bookstring:
    if character not in vocab_to_int:
        vocab_to_int[character] = count
        count += 1

    

# Add special tokens to vocab_to_int
codes = ['<PAD>','<EOS>','<GO>']
for code in codes:
    vocab_to_int[code] = count
    count += 1


filter_chars_dict = {}
for character in tqdm(bookstring):
    if character not in filter_chars_dict:
        filter_chars_dict[character] = 1
    else:
        filter_chars_dict[character] += 1

100%|██████████| 2864736/2864736 [00:01<00:00, 1692138.89it/s]


In [7]:
filter_list = []
for char in filter_chars_dict:
    if filter_chars_dict[char] < 55:
        filter_list.append(char)
re.sub(r"[',\s]","",str(filter_list))

'[]'

In [8]:
# Create another dictionary to convert integers to their respective characters
int_to_vocab = {}
for character, value in vocab_to_int.items():
    int_to_vocab[value] = character

In [9]:
# Split the text from the books into sentences.
sentences = []
for book in tqdm_notebook(clean_books, desc="1st loop"):
    for sentence in tqdm_notebook(book.split('. '), desc = "2nd loop", leave=False): #TODO: maybe split on ! ? and .?
        sentences.append(sentence + '.')
print("There are {} sentences.".format(len(sentences)))


There are 24218 sentences.


In [10]:
int_sentences = []


for sentence in sentences:
    int_sentence = []
    for character in sentence:
        int_sentence.append(vocab_to_int[character])
    int_sentences.append(int_sentence)


In [11]:
# Find the length of each sentence
lengths = []
for sentence in int_sentences:
    lengths.append(len(sentence))
lengths = pd.DataFrame(lengths, columns=["counts"])

lengths.describe()

,counts
count,24218.000000
mean,117.289867
std,97.195532
min,1.000000
25%,47.000000
50%,104.000000
75%,165.750000
max,3370.000000


In [12]:
# Limit the data we will use to train our model
max_length = 104
min_length = 10

good_sentences = []

for sentence in int_sentences:
    if len(sentence) <= max_length and len(sentence) >= min_length:
        good_sentences.append(sentence)

print("We will use {} to train and test our model.".format(len(good_sentences)))

We will use 10140 to train and test our model.


In [13]:
# Find the length of each good sentence
lengths = []
for sentence in good_sentences:
    lengths.append(len(sentence))
lengths = pd.DataFrame(lengths, columns=["counts"])

lengths.describe()

,counts
count,10140.000000
mean,56.294280
std,28.317424
min,10.000000
25%,31.000000
50%,57.000000
75%,81.000000
max,104.000000


In [14]:
# Split the data into training and testing sentences
training, testing = train_test_split(good_sentences, test_size = 0.15, random_state = 2)

print("Number of training sentences:", len(training))
print("Number of testing sentences:", len(testing))

Number of training sentences: 8619
Number of testing sentences: 1521


In [15]:
# Sort the sentences by length to reduce padding, which will allow the model to train faster
y_train = []
y_test = []

for i in range(min_length, max_length+1):
    for sentence in training:
        if len(sentence) == i:
            y_train.append(sentence)
    for sentence in testing:
        if len(sentence) == i:
            y_test.append(sentence)

In [17]:
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
           'n','o','p','q','r','s','t','u','v','w','x','y','z',]

def noise_maker(sentence, threshold):
    '''Relocate, remove, or add characters to create spelling mistakes'''
    
    noisy_sentence = []
    i = 0
    while i < len(sentence):
        random = np.random.uniform(0,1,1)
        # Most characters will be correct since the threshold value is high
        if random < threshold:
            noisy_sentence.append(sentence[i])
        else:
            new_random = np.random.uniform(0,1,1)
            # ~33% chance characters will swap locations
            if new_random > 0.67:
                if i == (len(sentence) - 1):
                    # If last character in sentence, it will not be typed
                    continue
                else:
                    # if any other character, swap order with following character
                    noisy_sentence.append(sentence[i+1])
                    noisy_sentence.append(sentence[i])
                    i += 1
            # ~33% chance an extra lower case letter will be added to the sentence
            elif new_random < 0.33:
                random_letter = np.random.choice(letters, 1)[0]
                noisy_sentence.append(vocab_to_int[random_letter])
                noisy_sentence.append(sentence[i])
            # ~33% chance a character will not be typed
            else:
                pass     
        i += 1
    return noisy_sentence

In [18]:
# Check to ensure noise_maker is making mistakes correctly.
threshold = 0.9
for sentence in y_train[:5]:
    print(sentence)
    print(noise_maker(sentence, threshold))
    print()

[26, 0, 1, 2, 3, 31, 0, 22, 60, 35]
[26, 0, 1, 2, 3, 31, 0, 22, 60, 35]

[41, 3, 9, 11, 29, 5, 14, 0, 14, 35]
[41, 3, 9, 11, 29, 5, 14, 0, 14, 35]

[62, 39, 14, 21, 11, 5, 29, 0, 54, 35]
[62, 39, 14, 21, 11, 5, 29, 0, 54, 35]

[53, 55, 38, 38, 49, 20, 26, 0, 53, 35]
[53, 55, 38, 38, 49, 20, 3, 26, 0, 53, 35]

[15, 18, 18, 22, 10, 12, 7, 3, 10, 35]
[15, 18, 18, 22, 10, 12, 7, 3, 10, 35]



In [19]:
x_train = []
for sentence in y_train:
    x_train.append(noise_maker(sentence, threshold))

In [20]:
x_test = []
for sentence in y_test:
    x_test.append(noise_maker(sentence, threshold))

In [21]:
save_pickle(x_test,"Data/","x_test")
save_pickle(x_train,"Data/","x_train")
save_pickle(y_test,"Data/","y_test")
save_pickle(y_train,"Data/","y_train")